# Model Size & Report Graph

<br>

### Model Size

In [1]:
import os 
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [12]:
file_path = './output/' + 'sst-2/' + 'quant/' + 'pytorch_model.bin' 
model = torch.load(file_path)

In [13]:
file_path = './output/' + 'sst-2/' + 'quant/' + 'pytorch_model.bin' 
model = torch.load(file_path)

In [22]:
num = 0 
param_size = 0
buffer_size = 0
for key, val in model.items():
    num += 1
    print("key:", key)
    print("val.nelement:", val.nelement())
    print("val.element_size:", val.element_size())
    print()
#  param_size += val.nelement() * val.element_size()

key: bert.embeddings.word_embeddings.weight
val.nelement: 23440896
val.element_size: 4

key: bert.embeddings.word_embeddings.weight_clip_val
val.nelement: 2
val.element_size: 4

key: bert.embeddings.position_embeddings.weight
val.nelement: 393216
val.element_size: 4

key: bert.embeddings.token_type_embeddings.weight
val.nelement: 1536
val.element_size: 4

key: bert.embeddings.LayerNorm.weight
val.nelement: 768
val.element_size: 4

key: bert.embeddings.LayerNorm.bias
val.nelement: 768
val.element_size: 4

key: bert.encoder.layer.0.attention.self.clip_query
val.nelement: 2
val.element_size: 4

key: bert.encoder.layer.0.attention.self.clip_key
val.nelement: 2
val.element_size: 4

key: bert.encoder.layer.0.attention.self.clip_value
val.nelement: 2
val.element_size: 4

key: bert.encoder.layer.0.attention.self.clip_attn
val.nelement: 2
val.element_size: 4

key: bert.encoder.layer.0.attention.self.query.weight
val.nelement: 589824
val.element_size: 4

key: bert.encoder.layer.0.attention.self.

In [18]:
param_size = 0
buffer_size = 0
for key, val in model.items():
    param_size += val.nelement() * val.element_size()

# for buffer in model.buffers():
    # buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size) / 1024**2
print('Size: {:.3f} MB'.format(size_all_mb))

Size: 417.649 MB


In [23]:
def get_bitwidth(tensor):
    """
    A function to return the smallest bitwidth that can represent the tensor.
    Specifically, if the max value of the tensor is smaller than 2 ** k - 1,
    then the tensor can be packed with k bit.
    """
    max_bitwidth = 8
    max_value = tensor

    for k in range(1, max_bitwidth + 1):
        if max_value < 2 ** k - 1:
            return k

    print("Error: The tensor is more than 8-bit")
    return max_bitwidth

In [24]:
get_bitwidth(3)

3

In [36]:
def find_optimal_compress_dim(tensor_shape, bit):
    """
    A function to find the index of optimal dimemsion to apply packing on.

    """
    index = 0
    curr_index = 0
    max_dim = 0
    for dim in tensor_shape:
        if (dim * bit) % 8 == 0:
            index = curr_index
            break
        if dim > max_dim:
            index = curr_index
            max_dim = dim
        curr_index += 1
    return index

def pack_tensor(tensor, bit):
    """
    A function to pack tensor into a packed_tensor with only one dimension difference.
    binary_tensor is the binary counterpart of tensor.
    padded_binary_tensor is a vector reshaped from binary_tensor, it is padded with zeros
    to have the same amount of values as the target binary_packed_tensor.
    binary_packed_tensor is the binary counterpart of packed_tensor.
    packed_tensor is the output tensor.

    """
    tmp_tensor_shape = list(tensor.shape)
    tmp_tensor_shape.append(1)
    tmp_tensor = np.zeros(tuple(tmp_tensor_shape), dtype=np.uint8)
    tmp_tensor[..., 0] = tensor

    binary_tensor = np.unpackbits(tmp_tensor, axis=-1)
    binary_tensor_shape = tmp_tensor_shape
    binary_tensor_shape[-1] = 8

    i = find_optimal_compress_dim(tensor.shape, bit)
    packed_dim = int(tensor.shape[i] * bit / 8) + (tensor.shape[i] * bit % 8 > 0)
    binary_packed_tensor_shape = binary_tensor_shape
    binary_packed_tensor_shape[i] = packed_dim

    packed_tensor = np.zeros(tuple(binary_packed_tensor_shape), dtype=np.uint8)
    padded_binary_tensor = binary_tensor[..., 8-bit: 8].reshape((-1))
    padding_width = packed_tensor.size - padded_binary_tensor.size
    padded_binary_tensor = np.pad(padded_binary_tensor, (0, padding_width), 'constant', constant_values=(0, 0))
    binary_packed_tensor = padded_binary_tensor.reshape(tuple(binary_packed_tensor_shape))

    packed_tensor = np.packbits(binary_packed_tensor, axis=-1)
    packed_tensor = packed_tensor[..., 0]

    return packed_tensor

In [53]:
max_value = 2
bit = 1
assert max_value <= 2 ** bit

tensor = (2*np.random.randint(max_value, size=(68, 68, 7, 7), dtype=np.int8)-1)
print(tensor[0, 0, :, :])
print(tensor.shape)

packed_tensor = pack_tensor(tensor, bit)
print(packed_tensor[0, 0, :, :])
print(packed_tensor.shape)

unpacked_tensor = unpack_tensor(packed_tensor, bit, (68, 68, 7, 7))
print(unpacked_tensor[0, 0, :, :])
print(unpacked_tensor.shape)

[[-1 -1 -1  1 -1 -1  1]
 [-1  1 -1  1  1 -1 -1]
 [-1 -1 -1 -1  1 -1 -1]
 [ 1 -1 -1 -1  1 -1  1]
 [ 1 -1  1  1  1  1  1]
 [ 1  1  1 -1 -1  1 -1]
 [-1 -1 -1 -1  1 -1  1]]
(68, 68, 7, 7)
[[255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255]]
(9, 68, 7, 7)
[[1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]]
(68, 68, 7, 7)


In [42]:
def unpack_tensor(packed_tensor, bit, target_shape):
    """
    A function to unpack a packed_tensor into the original tensor with target_shape.
    binary_packed_tensor is the binary counterpart of packed_tensor.
    unpadded_binary_packed_tensor is a vector reshaped from binary_packed_tensor,
    it is unpadded to have the same amount of values as binary_unpacked_tensor.
    binary_unpacked_tensor is the binary counterpart of unpacked_tensor.
    unpacked_tensor is the output tensor, it should be the same as the original tensor.

    """
    tmp_packed_tensor_shape = list(packed_tensor.shape)
    tmp_packed_tensor_shape.append(1)
    tmp_packed_tensor = np.zeros(tuple(tmp_packed_tensor_shape), dtype=np.uint8)
    tmp_packed_tensor[..., 0] = packed_tensor

    binary_packed_tensor = np.unpackbits(tmp_packed_tensor, axis=-1).reshape((-1))
    unpading_mask = np.arange(binary_packed_tensor.size - np.prod(target_shape) * bit) + np.prod(target_shape) * bit
    unpadded_binary_packed_tensor = np.delete(binary_packed_tensor, unpading_mask)

    binary_unpacked_tensor_shape = list(target_shape)
    binary_unpacked_tensor_shape.append(bit)
    unpadded_binary_packed_tensor = unpadded_binary_packed_tensor.reshape(tuple(binary_unpacked_tensor_shape))
    binary_unpacked_tensor_shape[-1] = 8

    binary_unpacked_tensor = np.zeros(tuple(binary_unpacked_tensor_shape), dtype=np.uint8)
    binary_unpacked_tensor[..., 8-bit: 8] = unpadded_binary_packed_tensor[..., :]

    unpacked_tensor = np.packbits(binary_unpacked_tensor, axis=-1).reshape(target_shape)

    return unpacked_tensor

In [33]:
num = 0
param_size = 0
buffer_size = 0
for key, val in model.items():
    num += 1
    param_size += val.nelement() * val.element_size()
    if num == 1:
        tensor = val

In [ ]:
unpacked_tensor = unpack_tensor(packed_tensor, 1)
unpacked_tensor

In [ ]:
param_size = 0
buffer_size = 0
for key, val in model['model_state_dict'].items():
    param_size += val.nelement() * val.element_size()

# for buffer in model.buffers():
    # buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size) / 1024**2
print('Size: {:.3f} MB'.format(size_all_mb))

In [ ]:
for key, val in model['model_state_dict'].items():
    print(val)

In [8]:
with open("parameters.txt", "a") as f:
    for key, val in model['model_state_dict'].items():
        f.write(key + "\n")

In [16]:
with open("model.txt", "a") as f:
    for k, v in model.items():
        source_list = []
        source_list.append("\n")
        source_list.append(str(k))
        source_list.append(str(v))
        f.write("\n".join(source_list))    

In [8]:
with open("model2.txt", "a") as f:
    for k, v in model.items():
        source_list = []
        source_list.append("\n")
        source_list.append(str(k))
        source_list.append(str(v))
        f.write("\n".join(source_list))      

In [20]:
def get_size(path):
        size = os.path.getsize(path)
        if size < 1024:
            return f"{size} bytes"
        elif size < pow(1024,2):
            return f"{round(size/1024, 2)} KB"
        elif size < pow(1024,3):
            return f"{round(size/(pow(1024,2)), 2)} MB"
        elif size < pow(1024,4):
            return f"{round(size/(pow(1024,3)), 2)} GB"

model_name = [model.replace('_log_df', '') for model in model_name]
for model in model_name:
    model_path = 'vit_cifar10/log_checkpoint/' + model + '/saves/' + model + '.49.pth.tar'
    model_size = get_size(model_path) 
    print('File Path:', model_path)
    print('File Size:', model_size)

File Path: vit_cifar10/log_checkpoint/vit_cifar10/saves/vit_cifar10.49.pth.tar
File Size: 41.12 MB
File Path: vit_cifar10/log_checkpoint/quant2_vit_cifar10/saves/quant2_vit_cifar10.49.pth.tar
File Size: 41.12 MB
File Path: vit_cifar10/log_checkpoint/quant4_vit_cifar10/saves/quant4_vit_cifar10.49.pth.tar
File Size: 41.12 MB
File Path: vit_cifar10/log_checkpoint/quant8_vit_cifar10/saves/quant8_vit_cifar10.49.pth.tar
File Size: 41.12 MB


In [26]:
model = "all_quant8_vit_cifar10"
model_path = 'vit_cifar10/log_checkpoint/' + model + '/saves/' + model + '.0.pth.tar'
model_size = get_size(model_path) 
print('File Path:', model_path)
print('File Size:', model_size)

File Path: vit_cifar10/log_checkpoint/all_quant8_vit_cifar10/saves/all_quant8_vit_cifar10.0.pth.tar
File Size: 41.12 MB
